In [ ]:
import psycopg2

API_KEY = '2CDC7D3628A900A6E59E9BC19A8E0B62'

try:
    connection = psycopg2.connect(
        host="26.84.71.27",       
        database="DataScienceProj",
        user="postgres",     
        password="SteamDB",    
        port="5432",
        options="-c client_encoding=UTF8"  
    )

    cursor = connection.cursor()

    cursor.execute("SELECT version();")
    version = cursor.fetchone()
    print("Versão do PostgreSQL:", version)

except Exception as error:
    print("Erro ao conectar ao banco de dados:", str(error).encode('utf-8', errors='replace').decode())


In [ ]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

connection = psycopg2.connect(
       host="26.84.71.27",       
       database="DataScienceProj",
       user="postgres",     
       password="SteamDB",    
       port="5432",
       options="-c client_encoding=UTF8"  
)

cursor = connection.cursor()

query = """
SELECT DISTINCT id, 
       (SELECT description 
        FROM steam.games_categories_steam cat 
        WHERE cat.id = a.id LIMIT 1) AS description, 
       qtdejogos 
FROM (
    SELECT DISTINCT id, 
           gcs.description, 
           COUNT(appid) OVER (PARTITION BY gcs.id) AS qtdejogos
    FROM steam.games_categories_steam gcs
) AS a 
ORDER BY qtdejogos DESC
"""
data = pd.read_sql_query(query, connection)

connection.close()
plt.figure(figsize=(12, 8))
sns.barplot(data=data, x='qtdejogos', y='description', palette='viridis')
plt.title("Quantidade de Jogos por Categoria", fontsize=16)
plt.xlabel("Quantidade de Jogos", fontsize=14)
plt.ylabel("Categoria", fontsize=14)
plt.xlim(0, data['qtdejogos'].max() + 5) 

plt.tight_layout()

plt.show()

In [ ]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

connection = psycopg2.connect(
    host="26.84.71.27",       
    database="DataScienceProj",
    user="postgres",     
    password="SteamDB",    
    port="5432",
    options="-c client_encoding=UTF8"  
)

cursor = connection.cursor()


if connection:
    query = """
    SELECT GI.NAME, 
           G.last_week_rank, 
           (cast(G.peak_in_game as float) / 1000) as peak_in_game
    FROM STEAM.GAMES G
    JOIN STEAM.GAME_INFO GI ON G.APPID = GI.steam_appid
    ORDER BY RANK
    """

    try:
        data = pd.read_sql_query(query, connection)
        connection.close()
        print("Conexão encerrada.")

        if data.empty:
            print("O DataFrame está vazio. Verifique sua consulta SQL.")
        else:
            print(data.head())  

            data.columns = [col.strip().upper() for col in data.columns]

            plt.figure(figsize=(12, 18))
            sns.barplot(data=data, x='PEAK_IN_GAME', y='NAME', palette='viridis')

            for index, value in enumerate(data['PEAK_IN_GAME']):
                plt.text(value, index, f"{value:.1f}", va='center', fontsize=10)

            plt.title("Jogos com Maior Número de Jogadores em Pico (a cada mil jogadores)", fontsize=16)
            plt.xlabel("Pico de Jogadores (em milhares)", fontsize=12)
            plt.ylabel("Nome do Jogo", fontsize=12)


            max_peak = data['PEAK_IN_GAME'].max()
            plt.xlim(0, max_peak * 1.1)  

            plt.tight_layout()


            plt.show()
    except Exception as e:
        print(f"Erro ao executar a consulta ou gerar o gráfico: {e}")


In [ ]:

import psycopg2
import matplotlib.pyplot as plt
from matplotlib.table import Table


query = """SELECT 'Gratuitos' AS tipo_jogo, COUNT(*) AS total, '-' as media_preço
FROM steam.game_info
WHERE is_free = true
UNION ALL
SELECT 'Pagos' AS tipo_jogo, COUNT(*) AS total, '-' as media_preço
FROM steam.game_info gi
WHERE is_free = false
union all 
select 'Pagos (com preço)' AS tipo_jogo, COUNT(*) AS total, to_char((select avg(initial) from steam.game_prices gp ), '000.00') as media_preco
from steam.game_info gi 
join steam.game_prices gp on gp.appid =gi.steam_appid 
;
"""

try:

    connection = psycopg2.connect(
        host="26.84.71.27",       
        database="DataScienceProj",
        user="postgres",     
        password="SteamDB",    
        port="5432",
        options="-c client_encoding=UTF8"  
    )

    cursor = connection.cursor()

    cursor.execute(query)

    results = cursor.fetchall()

    headers = ["Tipo de Jogo", "Total", "Média de Preço (USD)"]
    data = results


    fig, ax = plt.subplots(figsize=(6, 4))
    ax.axis("off")  


    table = Table(ax, bbox=[0, 0, 1, 1]) 
    cell_colors = [["#f0f0f0", "#f0f0f0"]] + [["#ffffff", "#ffffff"] for _ in data]


    for i, header in enumerate(headers):
        table.add_cell(0, i, width=0.5, height=0.2, text=header, loc="center", facecolor="#cccccc")


    for row_idx, row in enumerate(data, start=1):
        for col_idx, cell in enumerate(row):
            table.add_cell(row_idx, col_idx, width=0.5, height=0.2, text=str(cell), loc="center", facecolor="#ffffff")


    ax.add_table(table)


    plt.savefig("tabela_jogos.png", dpi=300)
    print("Tabela salva como 'tabela_jogos.png'.")

except Exception as e:
    print("Erro ao conectar ou executar a consulta:", e)
finally:

    if cursor:
        cursor.close()
    if connection:
        connection.close()

In [ ]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


connection = psycopg2.connect(
    host="26.84.71.27",
    database="DataScienceProj",
    user="postgres",
    password="SteamDB",
    port="5432",
    options="-c client_encoding=UTF8"
)

query_free = """
SELECT DISTINCT id, 
       (SELECT description 
        FROM steam.games_categories_steam cat 
        WHERE cat.id = a.id LIMIT 1) AS description, 
       qtdejogos 
FROM (
    SELECT DISTINCT gcs.id, 
           gcs.description, 
           COUNT(appid) OVER (PARTITION BY gcs.id) AS qtdejogos
    FROM steam.games_categories_steam gcs
    join steam.game_info gi on gi.steam_appid = gcs.appid
    where gi.is_free = true
) AS a 
ORDER BY qtdejogos desc
"""

query_pay = """
SELECT DISTINCT id, 
       (SELECT description 
        FROM steam.games_categories_steam cat 
        WHERE cat.id = a.id LIMIT 1) AS description, 
       qtdejogos 
FROM (
    SELECT DISTINCT gcs.id, 
           gcs.description, 
           COUNT(appid) OVER (PARTITION BY gcs.id) AS qtdejogos
    FROM steam.games_categories_steam gcs
    join steam.game_info gi on gi.steam_appid = gcs.appid
    where gi.is_free = false
) AS a 
ORDER BY qtdejogos desc
"""


data_free = pd.read_sql_query(query_free, connection)
data_pay = pd.read_sql_query(query_pay, connection)


connection.close()

plt.figure(figsize=(16, 8))

plt.subplot(1, 2, 1)
sns.barplot(data=data_free, x='qtdejogos', y='description', palette='Blues_r')
plt.title("Jogos Gratuitos por Categoria", fontsize=16)
plt.xlabel("Quantidade de Jogos", fontsize=14)
plt.ylabel("Categoria", fontsize=14)

plt.subplot(1, 2, 2)
sns.barplot(data=data_pay, x='qtdejogos', y='description', palette='Reds_r')
plt.title("Jogos Pagos por Categoria", fontsize=16)
plt.xlabel("Quantidade de Jogos", fontsize=14)
plt.ylabel("Categoria", fontsize=14)

plt.tight_layout()
plt.show()


In [ ]:
import psycopg2
import matplotlib.pyplot as plt
from matplotlib.table import Table

query = """
SELECT DISTINCT 
       (SELECT description 
        FROM steam.games_categories_steam cat 
        WHERE cat.id = gcs.id LIMIT 1) AS description, 
        avg(gp.initial) OVER (PARTITION BY gcs.id) preco_medio
FROM steam.games_categories_steam gcs
JOIN steam.game_info gi ON gi.steam_appid = gcs.appid
JOIN steam.game_prices gp ON gp.appid = gi.steam_appid
ORDER BY preco_medio DESC
"""

try:
    connection = psycopg2.connect(
        host="26.84.71.27",       
        database="DataScienceProj",
        user="postgres",     
        password="SteamDB",    
        port="5432",
        options="-c client_encoding=UTF8"  
    )

    cursor = connection.cursor()

    cursor.execute(query)

    results = cursor.fetchall()
    headers = ["Categoria", "Média de Preço (USD)"]
    data = results

   
    fig, ax = plt.subplots(figsize=(8,12))
    ax.axis("off") 

    table = Table(ax, bbox=[0, 0, 1, 1])  

    for i, header in enumerate(headers):
        table.add_cell(0, i, width=0.5, height=0.2, text=header, loc="center", facecolor="#cccccc")

    for row_idx, row in enumerate(data, start=1):
        description, preco_medio = row
        table.add_cell(row_idx, 0, width=0.5, height=0.2, text=str(description), loc="center", facecolor="#ffffff")
        table.add_cell(row_idx, 1, width=0.5, height=0.2, text=f"{preco_medio:.2f}", loc="center", facecolor="#ffffff")

    ax.add_table(table)

    plt.savefig("tabela_preco_categoria.png", dpi=300)
    print("Tabela salva como 'tabela_jogos.png'.")

except Exception as e:
    print("Erro ao conectar ou executar a consulta:", e)
finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()


In [ ]:

import psycopg2
import matplotlib.pyplot as plt
from matplotlib.table import Table


query = """
select country_name, '$'|| round(m_salary,2) as salary from public.country c 
join public.populous_countries pc on pc.country_id =c.id
join salary.country_salary cs on cs.country_id = c.id 
"""

try:
    connection = psycopg2.connect(
        host="26.84.71.27",       
        database="DataScienceProj",
        user="postgres",     
        password="SteamDB",    
        port="5432",
        options="-c client_encoding=UTF8"  
    )

    cursor = connection.cursor()

    cursor.execute(query)

    results = cursor.fetchall()

    headers = ["Pais por População", "Salário"]
    data = results

    
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.axis("off")  

    
    table = Table(ax, bbox=[0, 0, 1, 1])  
    cell_colors = [["#f0f0f0", "#f0f0f0"]] + [["#ffffff", "#ffffff"] for _ in data]

    for i, header in enumerate(headers):
        table.add_cell(0, i, width=0.5, height=0.2, text=header, loc="center", facecolor="#cccccc")

    
    for row_idx, row in enumerate(data, start=1):
        for col_idx, cell in enumerate(row):
            table.add_cell(row_idx, col_idx, width=0.5, height=0.2, text=str(cell), loc="center", facecolor="#ffffff")

    
    ax.add_table(table)

    
    plt.savefig("salario pais por populacao.png", dpi=300)
    print("Tabela salva como 'tabela_jogos.png'.")

except Exception as e:
    print("Erro ao conectar ou executar a consulta:", e)
finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()

In [ ]:

import psycopg2
import matplotlib.pyplot as plt
from matplotlib.table import Table


query = """
select country_name, '$'||round(m_salary,2) as salary from public.country c 
join public.best_country_live bcl on bcl.country_id = c.id
join salary.country_salary cs on cs.country_id = c.id 
"""

try:
    
    connection = psycopg2.connect(
        host="26.84.71.27",       
        database="DataScienceProj",
        user="postgres",     
        password="SteamDB",    
        port="5432",
        options="-c client_encoding=UTF8"  
    )

    cursor = connection.cursor()

    cursor.execute(query)

    results = cursor.fetchall()
    
    headers = ["Pais por IDH", "Salário"]
    data = results

    
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.axis("off")  

    table = Table(ax, bbox=[0, 0, 1, 1])  
    cell_colors = [["#f0f0f0", "#f0f0f0"]] + [["#ffffff", "#ffffff"] for _ in data]

    for i, header in enumerate(headers):
        table.add_cell(0, i, width=0.5, height=0.2, text=header, loc="center", facecolor="#cccccc")

    
    for row_idx, row in enumerate(data, start=1):
        for col_idx, cell in enumerate(row):
            table.add_cell(row_idx, col_idx, width=0.5, height=0.2, text=str(cell), loc="center", facecolor="#ffffff")

    
    ax.add_table(table)

    plt.savefig("salario pais por idh.png", dpi=300)
    print("Tabela salva como 'tabela_jogos.png'.")

except Exception as e:
    print("Erro ao conectar ou executar a consulta:", e)
finally:
    
    if cursor:
        cursor.close()
    if connection:
        connection.close()  

In [ ]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

connection = psycopg2.connect(
    host="26.84.71.27",
    database="DataScienceProj",
    user="postgres",
    password="SteamDB",
    port="5432"
)


query = """WITH country_salary_data AS (
    -- Busca os salários médios por país
    SELECT 
        c.country_name, 
        c.id AS country_id, 
        cs.m_salary,
        pc.id as country_rank
    FROM 
        public.country c
        JOIN public.populous_countries pc ON pc.country_id = c.id        
		join salary.country_salary cs on cs.country_id = c.id 
),
game_data AS (
    -- Busca os jogos pagos e seus preços
    SELECT 
        gi.name AS game_name, 
        gp.initial AS game_price, 
        g.rank,
        gp.appid
    FROM 
        steam.games g
        JOIN steam.game_info gi ON gi.steam_appid = g.appid
        JOIN steam.game_prices gp ON gp.appid = gi.steam_appid
    WHERE 
        gi.is_free = false
    ORDER BY 
        g.rank
    LIMIT 10
)
SELECT 
    csd.country_name, 
    csd.m_salary, 
    gd.game_name, 
    gd.game_price,
    ROUND((gd.game_price / csd.m_salary) * 100, 2) AS price_percentage
FROM 
    country_salary_data csd
    CROSS JOIN game_data gd
ORDER BY 
csd.country_rank,
    csd.country_name, 
    gd.rank;
"""

df = pd.read_sql_query(query, connection)

connection.close()


print(df)


plt.figure(figsize=(14, 7))
sns.barplot(
    data=df, 
    x="country_name", 
    y="price_percentage", 
    hue="game_name",
    palette="viridis"
)
plt.title("Porcentagem do preço dos jogos em relação ao salário médio por país", fontsize=14)
plt.xlabel("País")
plt.ylabel("Porcentagem (%)")
plt.legend(title="Jogo")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

connection = psycopg2.connect(
    host="26.84.71.27",
    database="DataScienceProj",
    user="postgres",
    password="SteamDB",
    port="5432"
)

query = """WITH country_salary_data AS (
    -- Busca os salários médios por país
    SELECT 
        c.country_name, 
        c.id AS country_id, 
        cs.m_salary,
        pc.id as country_rank
    FROM 
        public.country c
        JOIN public.best_country_live pc ON pc.country_id = c.id        
		join salary.country_salary cs on cs.country_id = c.id 
),
game_data AS (
    -- Busca os jogos pagos e seus preços
    SELECT 
        gi.name AS game_name, 
        gp.initial AS game_price, 
        g.rank,
        gp.appid
    FROM 
        steam.games g
        JOIN steam.game_info gi ON gi.steam_appid = g.appid
        JOIN steam.game_prices gp ON gp.appid = gi.steam_appid
    WHERE 
        gi.is_free = false
    ORDER BY 
        g.rank
    LIMIT 10
)
SELECT 
    csd.country_name, 
    csd.m_salary, 
    gd.game_name, 
    gd.game_price,
    ROUND((gd.game_price / csd.m_salary) * 100, 2) AS price_percentage
FROM 
    country_salary_data csd
    CROSS JOIN game_data gd
ORDER BY 
csd.country_rank,
    csd.country_name, 
    gd.rank;
"""

df = pd.read_sql_query(query, connection)

connection.close()


print(df)


plt.figure(figsize=(14, 7))
sns.barplot(
    data=df, 
    x="country_name", 
    y="price_percentage", 
    hue="game_name",
    palette="viridis"
)
plt.title("Porcentagem do preço dos jogos em relação ao salário médio por país", fontsize=14)
plt.xlabel("País")
plt.ylabel("Porcentagem (%)")
plt.legend(title="Jogo")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:

import psycopg2
import matplotlib.pyplot as plt
from matplotlib.table import Table


query = """select gi."name", '$'|| round(gp.initial,2) salary  from steam.games g
join steam.game_info gi on gi.steam_appid = g.appid
join steam.game_prices gp on gp.appid =g.appid 
where is_free = false 
order by g."rank" 
limit 10
"""

try:
    
    connection = psycopg2.connect(
        host="26.84.71.27",       
        database="DataScienceProj",
        user="postgres",     
        password="SteamDB",    
        port="5432",
        options="-c client_encoding=UTF8"  
    )

    cursor = connection.cursor()

    cursor.execute(query)

    results = cursor.fetchall()
    headers = ["Nome", "Preço"]
    data = results

    
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.axis("off")  

    table = Table(ax, bbox=[0, 0, 1, 1])  
    cell_colors = [["#f0f0f0", "#f0f0f0"]] + [["#ffffff", "#ffffff"] for _ in data]

    
    for i, header in enumerate(headers):
        table.add_cell(0, i, width=0.5, height=0.2, text=header, loc="center", facecolor="#cccccc")

    
    for row_idx, row in enumerate(data, start=1):
        for col_idx, cell in enumerate(row):
            table.add_cell(row_idx, col_idx, width=0.5, height=0.2, text=str(cell), loc="center", facecolor="#ffffff")

    
    ax.add_table(table)

    
    plt.savefig("preço jogos.png", dpi=300)
    print("Tabela salva como 'tabela_jogos.png'.")

except Exception as e:
    print("Erro ao conectar ou executar a consulta:", e)
finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()  

In [ ]:

import psycopg2
import matplotlib.pyplot as plt
from matplotlib.table import Table


query = """
select 'windows' as plataforma, count(*) quantidade from steam.game_info gi 
where windows 
union all 
select 'Mac' as plataforma, count(*) quantidade from steam.game_info gi 
where Mac 
union all 
select 'Linux' as plataforma, count(*) quantidade from steam.game_info gi 
where linux 
union all 
select 'Apenas Windows' as plataforma, count(*) quantidade from steam.game_info gi 
where linux = false and mac = false and windows 
"""

try:
    
    connection = psycopg2.connect(
        host="26.84.71.27",       
        database="DataScienceProj",
        user="postgres",     
        password="SteamDB",    
        port="5432",
        options="-c client_encoding=UTF8"  
    )

    cursor = connection.cursor()

    cursor.execute(query)

    results = cursor.fetchall()
    
    headers = ["Sistema Operacional", "Qde Jogos"]
    data = results

    
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.axis("off") 

    
    table = Table(ax, bbox=[0, 0, 1, 1]) 
    cell_colors = [["#f0f0f0", "#f0f0f0"]] + [["#ffffff", "#ffffff"] for _ in data]

    
    for i, header in enumerate(headers):
        table.add_cell(0, i, width=0.5, height=0.2, text=header, loc="center", facecolor="#cccccc")

    
    for row_idx, row in enumerate(data, start=1):
        for col_idx, cell in enumerate(row):
            table.add_cell(row_idx, col_idx, width=0.5, height=0.2, text=str(cell), loc="center", facecolor="#ffffff")

    
    ax.add_table(table)

    plt.savefig("plataforma.png", dpi=300)
    print("Tabela salva como 'tabela_jogos.png'.")

except Exception as e:
    print("Erro ao conectar ou executar a consulta:", e)
finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()  